In [255]:
import json
import requests 
import numpy as np
import requests
from scipy.stats import linregress
import pprint as pprint
import pandas as pd
import datetime
import matplotlib.pyplot as plt
from citipy import citipy
# from api_keys import weather_api_key
# from api_keys import g_key
import time as time
#import wget as wget

In [256]:
file_to_load_1 = "../Resources/time_series_covid_19_confirmed.csv"
file_to_load_2 = "../Resources/time_series_covid_19_deaths.csv"
file_to_load_3 = "../Resources/time_series_covid19_recovered.csv"

In [257]:
covid_19_confirmed = pd.read_csv(file_to_load_1)
confirmed_df_long = pd.DataFrame(covid_19_confirmed)

covid_19_deaths = pd.read_csv(file_to_load_2)
deaths_df_long = pd.DataFrame(covid_19_deaths)

covid_19_recovered = pd.read_csv(file_to_load_1)
recovered_df_long = pd.DataFrame(covid_19_recovered)

In [258]:
covid_19_confirmed.columns

Index(['Province/State', 'Country/Region', 'Lat', 'Long', '1/22/20', '1/23/20',
       '1/24/20', '1/25/20', '1/26/20', '1/27/20',
       ...
       '4/23/21', '4/24/21', '4/25/21', '4/26/21', '4/27/21', '4/28/21',
       '4/29/21', '4/30/21', '5/1/21', '5/2/21'],
      dtype='object', length=471)

In [259]:
covid_19_confirmed.columns[4:]

Index(['1/22/20', '1/23/20', '1/24/20', '1/25/20', '1/26/20', '1/27/20',
       '1/28/20', '1/29/20', '1/30/20', '1/31/20',
       ...
       '4/23/21', '4/24/21', '4/25/21', '4/26/21', '4/27/21', '4/28/21',
       '4/29/21', '4/30/21', '5/1/21', '5/2/21'],
      dtype='object', length=467)

In [260]:
dates = covid_19_confirmed.columns[4:]
confirmed_df_long = covid_19_confirmed.melt(
    id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
    value_vars=dates, 
    var_name='Date', 
    value_name='Confirmed'
)
deaths_df_long = covid_19_deaths.melt(
    id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
    value_vars=dates, 
    var_name='Date', 
    value_name='Deaths'
)
recovered_df_long = covid_19_recovered.melt(
    id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
    value_vars=dates, 
    var_name='Date', 
    value_name='Recovered'
)

In [261]:
# to remove recovered data for Canada due to mismatch issue Canada recovered data is counted by Country-wise rather than Province/State-wise).
recovered_df_long = recovered_df_long[recovered_df_long['Country/Region']!='Canada']

In [262]:
# Merging confirmed_df_long and deaths_df_long
full_table = confirmed_df_long.merge(
  right=deaths_df_long, 
  how='left',
  on=['Province/State', 'Country/Region', 'Date', 'Lat', 'Long']
)
# Merging full_table and recovered_df_long
full_table = full_table.merge(
  right=recovered_df_long, 
  how='left',
  on=['Province/State', 'Country/Region', 'Date', 'Lat', 'Long']
)

In [263]:
# Date from sting to datetime
full_table['Date'] = pd.to_datetime(full_table['Date'])
full_table

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
0,NaN,Afghanistan,33.939110,67.709953,2020-01-22,0,0,0.0
1,NaN,Albania,41.153300,20.168300,2020-01-22,0,0,0.0
2,NaN,Algeria,28.033900,1.659600,2020-01-22,0,0,0.0
3,NaN,Andorra,42.506300,1.521800,2020-01-22,0,0,0.0
4,NaN,Angola,-11.202700,17.873900,2020-01-22,0,0,0.0
...,...,...,...,...,...,...,...,...
128420,NaN,Vietnam,14.058324,108.277199,2021-05-02,2962,35,2962.0
128421,NaN,West Bank and Gaza,31.952200,35.233200,2021-05-02,297638,3272,297638.0
128422,NaN,Yemen,15.552727,48.516388,2021-05-02,6341,1233,6341.0
128423,NaN,Zambia,-13.133897,27.849332,2021-05-02,91693,1253,91693.0


In [264]:
# NA's are in Recovered Lat and Long
full_table.isna().sum()

Province/State    88263
Country/Region        0
Lat                 934
Long                934
Date                  0
Confirmed             0
Deaths                0
Recovered          7472
dtype: int64

In [265]:
# to replace NA's with zero for Recovered Lat and Long
full_table['Recovered'] = full_table['Recovered'].fillna(0)
full_table['Lat'] = full_table['Lat'].fillna(0)
full_table['Lat'] = full_table['Lat'].fillna(0)

In [266]:
# Coronavirus cases reported from 3 cruise ships: Grand Princess, Diamond Princess and MS Zaandam. 
# Data need to be extracted and treated differently due to Province/State and Country/Region mismatch over time
ship_rows = full_table['Province/State'].str.contains('Grand Princess') | covid_19_confirmed['Province/State'].str.contains('Diamond Princess') | covid_19_confirmed['Country/Region'].str.contains('Diamond Princess') | covid_19_confirmed['Country/Region'].str.contains('MS Zaandam')
full_ship = full_table[ship_rows]

In [267]:
# to remove rows for Grand Princess, Diamond Princess, and MS Zaandam
# Data need to be extracted and treated differently due to Province/State and Country/Region mismatch over time
full_table = full_table[~(ship_rows)]

In [268]:
covid_19_confirmed.dtypes

Province/State     object
Country/Region     object
Lat               float64
Long              float64
1/22/20             int64
                   ...   
4/28/21             int64
4/29/21             int64
4/30/21             int64
5/1/21              int64
5/2/21              int64
Length: 471, dtype: object

In [269]:
# Active Case = confirmed - deaths - recovered
full_table = full_table.copy()
full_table['Active'] = ""

In [270]:
full_table['Active'] = full_table['Confirmed'] - full_table['Deaths'] - full_table['Recovered']

In [271]:
full_table.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Active
0,NaN,Afghanistan,33.93911,67.709953,2020-01-22,0,0,0.0,0.0
1,NaN,Albania,41.15330,20.168300,2020-01-22,0,0,0.0,0.0
2,NaN,Algeria,28.03390,1.659600,2020-01-22,0,0,0.0,0.0
3,NaN,Andorra,42.50630,1.521800,2020-01-22,0,0,0.0,0.0
4,NaN,Angola,-11.20270,17.873900,2020-01-22,0,0,0.0,0.0


In [272]:
full_grouped = full_table.copy()

In [273]:
full_grouped = full_table.groupby(['Date', 'Country/Region'])['Confirmed', 'Deaths', 'Recovered', 'Active'].sum().reset_index()

<ipython-input-273-b4242cc93c67>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  full_grouped = full_table.groupby(['Date', 'Country/Region'])['Confirmed', 'Deaths', 'Recovered', 'Active'].sum().reset_index()


In [274]:
full_grouped

,Date,Country/Region,Confirmed,Deaths,Recovered,Active
0,2020-01-22,Afghanistan,0,0,0.0,0.0
1,2020-01-22,Albania,0,0,0.0,0.0
2,2020-01-22,Algeria,0,0,0.0,0.0
3,2020-01-22,Andorra,0,0,0.0,0.0
4,2020-01-22,Angola,0,0,0.0,0.0
...,...,...,...,...,...,...
89657,2021-05-02,Vietnam,2962,35,2962.0,-35.0
89658,2021-05-02,West Bank and Gaza,297638,3272,297638.0,-3272.0
89659,2021-05-02,Yemen,6341,1233,6341.0,-1233.0
89660,2021-05-02,Zambia,91693,1253,91693.0,-1253.0


In [275]:
full_grouped.shape

(89662, 6)

In [276]:
full_grouped.dtypes

Date              datetime64[ns]
Country/Region            object
Confirmed                  int64
Deaths                     int64
Recovered                float64
Active                   float64
dtype: object

In [277]:
full_grouped["Date"].max()

Timestamp('2021-05-02 00:00:00')

In [278]:
full_grouped["Date"].min() - full_grouped["Date"].max()

Timedelta('-466 days +00:00:00')

In [279]:
indexed_df = full_grouped.set_index("Date").groupby("Country/Region").resample("M").sum()

In [281]:
indexed_df

Confirmed  Deaths  Recovered   Active
Country/Region Date                                             
Afghanistan    2020-01-31          0       0        0.0      0.0
               2020-02-29          6       0        6.0      0.0
               2020-03-31       1164      27     1164.0    -27.0
               2020-04-30      27074     890    27074.0   -890.0
               2020-05-31     225674    5007   225674.0  -5007.0
...                              ...     ...        ...      ...
Zimbabwe       2021-01-31     787416   23012   787416.0 -23012.0
               2021-02-28     984049   38753   984049.0 -38753.0
               2021-03-31    1132264   46556  1132264.0 -46556.0
               2021-04-30    1126565   46375  1126565.0 -46375.0
               2021-05-31      76541    3138    76541.0  -3138.0

[3264 rows x 4 columns]